#  Preparation

### Modules

In [1]:
using JuMP, Gurobi

### Solver

In [2]:
solver = GurobiSolver(Presolve = 1)
;

### Data

In [3]:
P = 15 # Number of planes
c = rand(P)    # Cost of delays for each plane
t = rand(P)    # Ideal arrival time
l = rand(P)  # Time spent on the runway
set_up = rand(P,P)# Set-up times: entry (p,q) is the minimum time required between the finish of plane p and the arrival of plane q
d_max = rand(P)*10 # Maximum allowable delay for each plane
;

### Pre-checks

In [4]:
assert(all(l.>=0))
assert(length(t)==P)
assert(length(l)==P)
assert(length(d_max)==P)
assert(size(set_up)==(P,P))

### Big M calculation

In [5]:
BigM = Array{Float64}(P,P)
for p = 1:P
    for q = 1:P
        BigM[p,q] = 2*d_max[p] + 2*d_max[q] + l[p]+l[q] + 2*abs(t[p]-t[q]) 
    end
end

We would like $M_{p,q} \geq W_{p,q}$.

$$\begin{align*}
    W_{p,q} &= \max\{L_{p,q},L_{q,p}\}\\
            &\leq |L_{p,q}| + |L_{q,p}|\\
            &= |e_p - a_q| + |e_q - a_p|\\
            &= |l_p + a_p - a_q| + |l_q + a_q - a_p|\\
            &\leq | l_p| + |a_p - a_q| + |l_q| + |a_q - a_p|&\text{By the triangle inequality}\\
            &= l_p +  l_q + 2| a_p - a_q|&\text{since }l\geq 0\\
            &= l_p +  l_q + 2| t_p+d_p-t_q-d_q|\\
            &\leq l_p +  l_q + 2| t_p-t_q| + 2|d_p-d_q|&\text{By the triangle inequality}\\
            &\leq 2|d_p|+2|d_q|+l_p +  l_q + 2| t_p-t_q|&\text{By the triangle inequality}\\
            &\leq 2d_{\max_p}+2d_{\max_q}+l_p +  l_q + 2| t_p-t_q|\\
            &=:M_{p,q}
\end{align*}$$

# Model

### Main model object

In [6]:
m = Model(solver = solver)
;

### Variables 

In [7]:
@variable(m, a[1:P]) # Scheduled arrival times
@variable(m, e[1:P]) # Scheduled ending times
@variable(m, d[1:P] >= 0) # Scheduled delays
@variable(m, L[1:P,1:P]) # Time between the first plane, p, finishing and the second, q, arriving
@variable(m, W[1:P,1:P] >= 0) # The smallest window of time that contains all intervals of time in which either plane p or plane q is landing.
@variable(m, G[1:P,1:P] >= 0) # The gap between plane p ad q's landings
@variable(m, A[1:P,1:P], Bin) # Binary matrix; entry (p,q) is equal to 1 if plane p arrives before plane q, and 0 otherwise.
;

### Objective

In [8]:
@objective(m, Min, dot(c,d)) # Minimise the total cost of delays
;

### Essential Constraints

In [9]:
@constraint(m, a .== t + d) # The arrival time is the ideal arrival time, plus delay
@constraint(m, e .== a + l) # The ending time is the arrival time plus time spent on runway
@constraint(m, d .<= d_max) # Maximum delay
for p = 1:P
    for q = 1:P
        @constraint(m, L[p,q] == e[p] - a[q]) # By the definition of L
        
        @constraint(m, W[p,q] >= L[p,q]) # W[p,q] == maximum(L[p,q], L[q,p]), constraint I
        @constraint(m, W[p,q] >= L[q,p]) # W[p,q] == maximum(L[p,q], L[q,p]), constraint II
        @constraint(m, W[p,q] <= L[p,q] + BigM[p,q]*A[p,q]) # W[p,q] == maximum(L[p,q], L[q,p]), constraint III
        @constraint(m, W[p,q] <= L[q,p] + BigM[p,q]*(1-A[p,q])) # W[p,q] == maximum(L[p,q], L[q,p]), constraint IV

        if p != q
            @constraint(m, G[p,q]+l[p]+l[q] == W[p,q]) # The window W is the time the two planes spend each on the runway, plus the gap between these planes.
            @constraint(m, G[p,q] >= set_up[p,q].*A[p,q]) # Enforce set-up times
            @constraint(m, A[p,q] + A[q,p] == 1) # Plane p comes before plane q, or q before p, but not both.
        end
    end
end

### Additional Constraints

Now for some constraints that should help make the problem easier to solve, but aren't necessary in enforcing the logic of the problem. The first is to get an upper bound on each delay. In the very worst case scenario, plane $p$ ideally arrives first, but gets scheduled to arrive last, after all the other planes have arrived one after the other, incurring times of
$$\sum_{i\neq p} l_i+(P-1)\max_{i,j}\mathrm{(Setup)}_{i,j}$$
in total. Therefore our constraint is

In [10]:
for p = 1:P
    @constraint(m, d[p] <= sum(l)-l[p] + (P-1)*maximum(set_up))
end

In [11]:
tic();
status = solve(m);
toc();

Academic license - for non-commercial use only
Optimize a model with 1815 rows, 945 columns and 4275 nonzeros
Variable types: 720 continuous, 225 integer (225 binary)
Coefficient statistics:
  Matrix range     [5e-03, 4e+01]
  Objective range  [3e-02, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e-02, 4e+01]
Presolve removed 660 rows and 542 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


elapsed time: 2.977388351 seconds


Given a sequence of planes $p,q,r,s...$, plane $p$ comes before all of them and so will have $P-1$ ones in its row of the $A$ matrix.
Plane $q$ has one fewer, $P-2$, and plane $r$ has $P-3$. This continues up until the final plane numbered $P$, which has all zeros.
The total number of ones in the $A$ matrix is therefore
$$\begin{align*}
    \sum_{p=1}^{P} (P-p) &= \sum_{p=1}^{P} P - \sum_{p=1}^{P} p\\
					   &= P^2 - P(P+1)/2\\
					   &= P(P-1)/2
\end{align*}$$

In [12]:
@constraint(m, sum(A)==P*(P-1)/2)
;

In [13]:
tic();
status = solve(m);
toc();

Optimize a model with 1816 rows, 945 columns and 4500 nonzeros
Variable types: 720 continuous, 225 integer (225 binary)
Coefficient statistics:
  Matrix range     [5e-03, 4e+01]
  Objective range  [3e-02, 9e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e-02, 1e+02]
Presolve removed 661 rows and 542 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
elapsed time: 0.044605608 seconds


"Coming before" is a transitive property. What this means is that if p comes before q, and q comes before r, then p comes before r. This fact can put some more constraints on the A matrix.

In [14]:
for p = 1:P
    for q = 1:P
        for r = 1:P
            @constraint(m, A[p,q] + A[q,r] <= 1 + A[p,r])
        end
    end
end
;

The idea behind this formulation is that if the planes land one after the other you will have a situation like this:

<img src="files/no_overlap.png"> 

whereas with overlap you have this:

<img src="files/yes_overlap.png"> 



 In the first case, $W_{p,q} \geq l_p + l_q$, while in the second, that inequality is violated. Since $G \geq 0$, this forces no overlap.
Notice also how $W_{p,q} = \max(L_{p,q}, L_{q,p})$. At least one of the two arguments must be positive, so $W$ must be too.